<a href="https://colab.research.google.com/github/RovisLab/Course_AI/blob/main/Lab_02_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np